In [1]:
import numpy as np
import matplotlib.pyplot as plt
import stock
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box
import xueqiu
import kline

In [2]:
"""装入一个分类的所有数据并对齐，返回一个大的矩阵"""
def LoadCategoryCompanys(categoryName):
    companys = stock.query("""select * from company_select where category='%s'"""%(categoryName))
    stock.closedb()
    C,K,D = stock.loadKline('SZ:399001') #和深圳成指对齐
    KS = np.zeros((len(companys),len(K),5))
    i = 0
    for c in companys:
        c,k,d = stock.loadKline(c[1])
        KS[i] = stock.alignK(D,k,d)
        i+=1

    return companys,KS,D

# 行业手动初选

In [3]:
"""
输入分类名称将分类下的全部股票列出
"""
def CategorySelect(name,output=widgets.Output()):
    
    def K(c,k,d):
        butsel = widgets.Checkbox(value=True if c[-1]==1 else False,
            description='选择',
            disabled=False,
            indent=False)
        
        def onClick(event):
            value = 1 if event['new'] else 0
            stock.execute("""update company_select set manual=%d where code='%s'"""%(value,c[1]))
            stock.closedb()
            
        butsel.observe(onClick,names='value')
        with output:
            display(butsel)
            #display(Markdown("""[%s %s ttm:%.2f pb:%.2f](https://xueqiu.com/S/%s)"""%(c[2],c[1],c[4],c[5],c[1])))
            kline.Plote(k,date=d,config={"index":True},companyInfo=c).show(len(k)-440,len(k),figsize=(32,15))    
    coms,ks,date = LoadCategoryCompanys(name)
    kvs = np.zeros((ks.shape[1]))
    for i in range(ks.shape[1]):
        kvs[i] = ks[:,i,0].sum()
    VE = stock.volumeEnergy(kvs,20)
    VEKDJ = stock.kdj(VE,9)
    for i in range(len(coms)):
        K(coms[i],ks[i],date)

"""
列出全部分类
点击不同分类列出该分类下的全部股票的图表
"""
def CategoryList():
    output = widgets.Output()
    categorys = stock.query("""select * from category""")
    stock.closedb()
    
    items_layout = Layout( width='auto')     # override the default width of the button to 'auto' to let the button grow

    box_layout = Layout(display='flex',
                        flex_flow='wrap',
                        align_items='stretch',
                        border='solid',
                        width='100%')

    items = []
    def onClick(e):
        output.clear_output()
        CategorySelect(e.description,output)
        
    for c in categorys:
        but = widgets.Button(
            description=c[1],
            disabled=False,
            button_style='',
            tooltip=c[1])
        but.on_click(onClick)
        items.append(but)
    box = Box(children=items, layout=box_layout)
    display(box,output)

In [4]:
CategoryList()

Box(children=(Button(description='半导体', style=ButtonStyle(), tooltip='半导体'), Button(description='化学纤维', style=…

Output()